# iModulonDB Tutorial

This notebook will help you add information and generate 

Follow the advice [here](https://github.com/SBRG/modulytics/wiki/Adding-a-Private-Project) to get started, and learn to use pymodulon with the initial Tutorial if needed.

Please contact krychel@eng.ucsd.edu with any questions!

In [1]:
import sys

from pymodulon.core import IcaData
from pymodulon.io import *
from pymodulon.imodulondb import *
import pandas as pd

## Step 1: Load Your Data

More details on this can be found in the main tutorial.

All that is absolutely required is the following:

- M matrix (part of initial ICA output)
- A matrix (part of initial ICA output)
- X matrix (log_tpm.csv) -- initial data, preferably not normalized to the base condition.
- Sample_table containing:
    - Either a column labeled 'sample' or an index (leftmost column) that serves the purpose of sample labels. These should be unique to each sample (no duplicates in the entire table), and it must match the columns of A and X.
    - A column labeled 'project' -- this is the largest grouping of samples; they may be downloaded from the same original paper or contain a set of samples/experiments with a common theme. The contents of this column will appear on the X axis when zoomed all the way out in the activity bar graphs on iModulonDB.
    - A column labeled 'condition' -- this groups samples such that all biological replicates have the same condition name. Samples in the same project with matching conditions will be grouped into a single bar in the activity bar graphs. If you'd like to re-use condition names across different projects (for example, giving each project a 'control' condition), that is okay.
    
Additional information will only improve your version of the site!

In [2]:
ica_data = load_json_model("../../mtb_ica.json")

In [3]:
ica_data.imodulon_table

,TF,Function,Category,n_genes,precision,recall,category_num
SigH,sigH,Sigma Factor involved in heat shock and oxidat...,Virulence/Persistence,45,0.082192,0.800000,0
Polyketide Synthase Complex,None,KEGG Map related to the synthesis of type II p...,Fatty Acid/Cholestrol,2,0.333333,1.000000,1
IniR,iniR,Transcription factor that regulates the expres...,Antibiotic,3,0.100000,1.000000,2
PyrR,pyrR,Transcription factor that regulates the expres...,Nucleic Acids,44,0.000000,0.000000,3
WhiB4,whiB4,Redox-Responsive transcription regulator that ...,Redox,95,0.184834,0.410526,4
...,...,...,...,...,...,...,...
Unc_9,None,Uncharacterized iModulon 9,Uncharacterized,24,0.000000,0.000000,75
SG_13,None,Single Gene iModulon 13,Single Gene,5,0.000000,0.000000,76
Central Carbon Metabolism,None,iModulon enriched with the Central Carbon Meta...,Central Carbon,28,0.000000,0.000000,77
Positive Regulation of Growth,None,iModulon enriched with the Positive Regulation...,Growth,26,0.366667,0.333333,78


In [4]:
ica_data.trn

,Unnamed: 0,regulator,regulator_id,evidence,gene_id
0,0,fhaB,Rv0019c,ChipSeq,Rv1733c
1,1,fhaA,Rv0020c,ChipSeq,Rv0020c
2,2,fhaA,Rv0020c,ChipSeq,Rv1733c
3,3,whiB5,Rv0022c,ChipSeq,Rv0013
4,4,whiB5,Rv0022c,ChipSeq,Rv0038
...,...,...,...,...,...
24300,24300,lysG,Rv1985c,Differential Expression,Rv2932
24301,24301,lysG,Rv1985c,Differential Expression,Rv2933
24302,24302,lysG,Rv1985c,Differential Expression,Rv2934
24303,24303,lysG,Rv1985c,Differential Expression,Rv1986


In [5]:
ica_data.M

,SigH,Polyketide Synthase Complex,IniR,PyrR,WhiB4,DevR-1,MarR,GroEL-GroES Complex,IdeR,SigK,...,Lsr2,PDIM;PGL Synthesis,Unc_8,SG_11,SG_12,Unc_9,SG_13,Central Carbon Metabolism,Positive Regulation of Growth,WhiB4/IdeR
Rv0001,-0.002362,-0.000345,0.018863,0.003751,-0.004513,-0.005628,-0.008293,-0.001941,0.004094,-0.003420,...,0.043023,0.021328,-0.001038,-0.006600,0.014561,0.000662,-0.001623,-0.023295,-0.006675,-0.006532
Rv0002,-0.005701,0.003088,-0.002460,0.004126,0.004209,-0.004811,-0.014755,-0.009028,0.002528,-0.004720,...,0.020308,0.005312,-0.020528,-0.007400,0.003026,-0.005617,0.000873,-0.005503,-0.006236,-0.010049
Rv0003,-0.015815,0.002336,0.009974,0.001325,0.005892,-0.010023,-0.003943,-0.013108,0.000750,-0.001532,...,0.026046,-0.000131,-0.005501,0.003283,0.002269,-0.001001,-0.003914,-0.004735,-0.003045,-0.011112
Rv0004,-0.010953,0.014130,0.005316,0.007447,0.002355,-0.005675,0.001232,0.002642,-0.002937,-0.009452,...,0.002391,-0.000943,0.047164,0.000554,-0.010171,-0.001388,-0.008658,-0.018899,0.004760,-0.013165
Rv0005,-0.009394,-0.002810,-0.012055,0.020868,-0.005547,-0.001030,-0.001101,0.003174,-0.003904,0.000795,...,-0.000266,-0.003107,0.000432,-0.003614,0.001742,-0.003190,0.006061,-0.010431,-0.001558,-0.010690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Rv3920c,-0.010412,0.013396,-0.011807,0.000782,-0.011104,-0.004583,-0.008442,-0.010533,-0.000110,-0.012717,...,0.004892,-0.008485,-0.000913,-0.001425,-0.002206,-0.002644,-0.007711,-0.019890,0.003186,0.000633
Rv3921c,-0.012022,0.005247,-0.011139,0.004236,-0.006812,-0.001795,-0.016140,-0.010954,0.001175,-0.006727,...,0.033010,-0.001548,0.000063,-0.000003,0.005848,0.004321,-0.001381,-0.022975,0.004075,-0.000829
Rv3922c,-0.009557,0.003253,0.005597,0.000352,-0.003766,-0.000583,-0.017275,0.002083,0.002723,0.000216,...,0.024754,0.001314,0.000270,0.014457,0.011590,0.001659,-0.002904,-0.009478,0.014821,-0.003982
Rv3923c,-0.002366,0.004769,-0.005973,0.000878,-0.007411,-0.003503,-0.015604,-0.003700,0.000320,-0.000618,...,0.015781,-0.005540,0.002152,0.003105,0.006952,0.009292,-0.012886,-0.007947,-0.005386,-0.002804


## Step 2: Fill out some metadata

When you load an ica_data model, it fills out default metadata for you. Therefore you can technically skip this step.

### 2.a: Splash Table (Dictionary)

This table provides the basic elements shown on [the splash page](http://imodulondb.org) and the names of the directories in which all data will be stored.

First, look at the default one:

In [6]:
ica_data.splash_table

{'large_title': 'New Dataset',
 'subtitle': 'Unpublished study',
 'author': 'Pymodulon User',
 'organism_folder': 'new_org',
 'dataset_folder': 'new_dataset'}

The first three entries are shown on the splash page directly. The last two are important folder names. Typical organism folder names are in the form 'g_species'. Choose a name for your dataset that will be unique; it is likely the only one for your organism, so you could choose 'modulome' if that is the project you are working on.

Let's update this table with your information.

In [7]:
# FILL IN THESE DETAILS

# <i> tags italicize the species name
# So far on the site, large titles are all simply species names.
ica_data.splash_table['large_title'] = '<i>M. tuberculosis</i>'

# subtitle provides additional information about the dataset's origins
ica_data.splash_table['subtitle'] = 'M. Tuberculosis Dataset'
ica_data.splash_table['author'] = 'Yoo, et al.'

# these will correspond to actual folders, which will be generated for you.
ica_data.splash_table['organism_folder'] = 'm_tuberculosis'
ica_data.splash_table['dataset_folder'] = 'modulome'

# make sure the output is desired
ica_data.splash_table

{'large_title': '<i>M. tuberculosis</i>',
 'subtitle': 'M. Tuberculosis Dataset',
 'author': 'Yoo, et al.',
 'organism_folder': 'm_tuberculosis',
 'dataset_folder': 'modulome'}

### 2.b: Dataset Table (Dictionary)

This table provides the basic elements shown in the [dataset page](https://imodulondb.org/dataset.html?organism=e_coli&dataset=precise1). The numerical elements are computed for you in the default.

If desired, you can add your own entries to this with whatever names you'd like. They can even include links if you obey the [rules for making HTML links](https://www.w3schools.com/html/html_links.asp). For example, the published datasets include a 'Publication' entry.

In [8]:
ica_data.dataset_table

{'Title': 'New Dataset',
 'Organism': 'New Organism',
 'Strain': 'Unknown Strain',
 'Number of Samples': 647,
 'Number of Unique Conditions': 231,
 'Number of Genes': 3906,
 'Number of iModulons': 80}

In [9]:
# FILL IN THESE DETAILS, TOO

# This title can combine some information from the splash table
ica_data.dataset_table['Title'] = 'M. tuberculosis Modulome'
ica_data.dataset_table['Organism'] = '<i>M. tuberculosis</i>'
ica_data.dataset_table['Strain'] = 'H37Rv'

# additional rows if desired
ica_data.dataset_table['Useful Additional Link'] = '<a href="https://github.com/Reosu/modulome_mtb">Github</a>'

# check output
ica_data.dataset_table

{'Title': 'M. tuberculosis Modulome',
 'Organism': '<i>M. tuberculosis</i>',
 'Strain': 'H37Rv',
 'Number of Samples': 647,
 'Number of Unique Conditions': 231,
 'Number of Genes': 3906,
 'Number of iModulons': 80,
 'Useful Additional Link': '<a href="https://github.com/Reosu/modulome_mtb">Github</a>'}

## Step 3: Add Links (Optional)

If your organism has a database like EcoCyc, SubtiWiki, or AureoWiki, you may be interested in adding links to it.

iModulonDB has two kinds of links:

- gene_links: 
    - Associate gene **loci** from the **index of ica_data.M** to websites
    - On the [gene pages](https://imodulondb.org/gene.html?organism=e_coli&dataset=precise1&gene_id=b0002), these links have the name of the database that you are linking to. The name of the database needs to be saved as a string in ica_data.link_database.
    - For programming reasons, the default value of gene_links is all genes as keys, each pointing to the value np.nan.
    
- tf_links:
    - Associate **TF names** from **ica_data.TRN.regulator and ica_data.imodulon_table.regulator** to websites
    - On the [iModulon pages](https://imodulondb.org/iModulon.html?organism=e_coli&dataset=precise1&k=43), the name of the regulator becomes a link. You never have to state the name of this database, and it doesn't have to be the same one used for gene_links (i.e. ica_data.link_database does NOT have to be the one you use).
    - For programming reasons, the default value is an empty dictionary.
    
Note that gene_links uses gene loci since not every gene has a readable name, but tf_links uses readable names since not all regulators have associated gene loci. This could create some confusion between the two types of links; for example, the gene b2741 in E. coli encodes the TF rpoS:

- ica_data.gene_links\['b2741'\] = 'https://ecocyc.org/gene?orgid=ECOLI&id=EG10510 '
- ica_data.tf_links\['rpoS'\] = 'http://regulondb.ccg.unam.mx/sigmulon?term=ECK125110294&organism=ECK12&format=jsp&type=sigmulon '

In this example, ica_data.link_database = 'EcoCyc' since that is where the gene links point to.

You are free to skip this step, or you can use any of the following strategies to generate your links:

#### Strategy 1: Look at the URLs in your database.

Search for genes in your database. You might notice that it always uses the same URL, but enters your search term in part of that URL. You could then take the pieces of that URL and use those to generate all needed URLs. Below is an example from psuedomonas aeruginosa:

In [10]:
prefix = 'https://mycobrowser.epfl.ch/genes/'
suffix = ''

temp_gene_links = dict()
for gene in ica_data.M.index:
    temp_gene_links[gene] = prefix + gene + suffix
    
temp_tf_links = dict()
for gene in ica_data.trn.regulator.unique():
    temp_tf_links[gene] = prefix + gene + suffix
    
# Then, test a few of the links and uncomment these lines

ica_data.gene_links = temp_gene_links
ica_data.tf_links = temp_tf_links

You may need to add a suffix in addition to a prefix. You may also have to deal with databases that use different loci abbreviations than the ones you have - this can be remedied by finding a mapping file between the two naming systems.

#### Strategy 2: Make a file

If you are more proficient in excel than python, have found a gene <-> link mapping file elsewhere, or already used strategy 1 to generate a file, you can make sure it has only two columns (gene and link) and no headers, then simply read it in as shown below.

This may be especially useful if you have no database, but want to keep a file of gene_links or tf_links with specific papers instead of db entries.

In [11]:
# ica_data.gene_links = 'gene_links.csv'

#### Strategy 3: Individual modifications

You can also just tell the model the links one at a time. This can be useful if you generate all links with another strategy, but want to overide it in a few cases.

In [12]:
# ica_data.gene_links['b2741'] = 'https://ecocyc.org/gene?orgid=ECOLI&id=EG10510'

#### If using gene links, don't forget to update ica_data.link_database

If you don't, the default value is 'External Database'. If you have no links, nothing will appear so this is irrelevant.

In [13]:
# ica_data.link_database = 'EcoCyc'

ica_data.link_database

'External Database'

## Step 4: Check for compatibility

Though the code cannot check for all edge cases and concerns, you can use the function 'imodulondb_compatibility()' to see what information you are missing, mostly through a check of the columns of your DataFrames. 

#### 4.a: Modify column names

First, let's see what we are missing:

In [14]:
imodulondb_compatibility(ica_data)

iModulon Table is missing a name column
iModulon Table is missing a Regulator column
Note that Regulator is displayed in the dataset page so it can have spelled out 'and' and 'or' operators.If you have a TF column, a Regulator column will begenerated for you (but not vice versa).

Gene Table is missing the optional length column

Sample Table is missing the optional sample column
The 0th column will be used to define the names of samples in the activity bar graph unless you add this column.


Now, let's see if we actually DO have the information that the function believes is missing -- it is likely that some of the names of the columns are simply not what iModulonDB is expecting. If so, we simply rename it to what is expected.

If you actually DO NOT have some of the information, that is okay -- after this step, we'll fill in the blanks with default values as needed.

In [15]:
ica_data.imodulon_table

,TF,Function,Category,n_genes,precision,recall,category_num
SigH,sigH,Sigma Factor involved in heat shock and oxidat...,Virulence/Persistence,45,0.082192,0.800000,0
Polyketide Synthase Complex,None,KEGG Map related to the synthesis of type II p...,Fatty Acid/Cholestrol,2,0.333333,1.000000,1
IniR,iniR,Transcription factor that regulates the expres...,Antibiotic,3,0.100000,1.000000,2
PyrR,pyrR,Transcription factor that regulates the expres...,Nucleic Acids,44,0.000000,0.000000,3
WhiB4,whiB4,Redox-Responsive transcription regulator that ...,Redox,95,0.184834,0.410526,4
...,...,...,...,...,...,...,...
Unc_9,None,Uncharacterized iModulon 9,Uncharacterized,24,0.000000,0.000000,75
SG_13,None,Single Gene iModulon 13,Single Gene,5,0.000000,0.000000,76
Central Carbon Metabolism,None,iModulon enriched with the Central Carbon Meta...,Central Carbon,28,0.000000,0.000000,77
Positive Regulation of Growth,None,iModulon enriched with the Positive Regulation...,Growth,26,0.366667,0.333333,78


In [16]:
# We do have regulators. Rename that column to TFs
# We also have n_genes. Rename that column to n_genes
rename_dict = {'regulator': 'TF', # this is more important than the 'Regulator' column
               'regulon_size': 'n_genes'} 
ica_data.imodulon_table = ica_data.imodulon_table.rename(rename_dict, axis = 1)

# We also have names. They were in the index, but the code wants them in the main dataframe as well.
# If you would rather not add this column, the code will do so automatically.
ica_data.imodulon_table['name'] = ica_data.imodulon_table.index

# We don't have a separate Regulator column. Don't worry about that!

In [17]:
ica_data.gene_table

,gene_name,accession,old_locus_tag,start,stop,strand,gene_product,COG,uniprot,operon,regulator
Rv0001,dnaA,NC_000962.3,NaN,1,1524,+,chromosomal replication initiator protein DnaA,"Replication, recombination and repair",P9WNW3,Op1,"Rv0023,Rv0047c,Rv0081,Rv0302,Rv0474,tcrA,Rv067..."
Rv0002,dnaN,NC_000962.3,NaN,2052,3260,+,DNA polymerase III subunit beta,"Replication, recombination and repair",P9WNU1,Op2,"Rv0047c,Rv0081,Rv0135c,Rv0324,Rv0767c,Rv0880,t..."
Rv0003,recF,NC_000962.3,NaN,3280,4437,+,DNA replication/repair protein RecF,"Replication, recombination and repair",P9WHI9,Op2,"Rv0023,Rv0081,Rv0302,trcR,Rv1353c,Rv1776c,mbcA..."
Rv0004,Rv0004,NC_000962.3,NaN,4434,4997,+,hypothetical protein,Function unknown,P9WFL1,Op2,"Rv0023,Rv0047c,Rv0324,Rv1473A,kstR"
Rv0005,gyrB,NC_000962.3,NaN,5240,7267,+,DNA gyrase subunit B,"Replication, recombination and repair",P9WG45,Op3,"Rv0023,Rv0047c,Rv0081,Rv0324,Rv0678,csoR,Rv104..."
...,...,...,...,...,...,...,...,...,...,...,...
Rv3920c,Rv3920c,NC_000962.3,NaN,4408334,4408897,-,hypothetical protein,Function unknown,O53598,Op2448,"Rv0047c,Rv0081,Rv0302,csoR,Rv1353c,cmr,Rv1719,..."
Rv3921c,Rv3921c,NC_000962.3,NaN,4408969,4410069,-,membrane protein insertase YidC,"Intracellular trafficking, secretion, and vesi...",P9WIT5,Op2449,"Rv0023,Rv0047c,Rv0081,Rv0135c,Rv0302,Rv0324,cs..."
Rv3922c,Rv3922c,NC_000962.3,NaN,4410053,4410415,-,membrane protein insertion efficiency factor,Function unknown,P9WFL9,Op2449,"Rv2034,lsr2"
Rv3923c,rnpA,NC_000962.3,NaN,4410412,4410789,-,ribonuclease P protein component,"Translation, ribosomal structure and biogenesis",P9WGZ3,Op2449,"Rv0081,Rv0678,Rv1049,Rv1255c,Rv2324,lsr2,tcrX,..."


In [18]:
# we have the "missing" cog column:
rename_dict = {'cog': 'COG'}
ica_data.gene_table = ica_data.gene_table.rename(rename_dict, axis = 1)

# we don't have the gene_product column. This is a nice-to-have, but you can ignore it.

In [19]:
ica_data.sample_table

,ReleaseDate,TaxID,ScientificName,species_tax_id,class_tax_id,strain_tax_id,species,LibraryLayout,Platform,Model,...,condition,Biological Replicates,reference_condition,passed_fastqc,passed_reads_mapped_to_CDS,passed_global_correlation,full_name,passed_similar_replicates,passed_number_replicates,DOI
SRX3995807,2018-04-29 10:56:12,83332,Mycobacterium tuberculosis H37Rv,1773,1760,83332,Mycobacterium tuberculosis,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,AX_10_MIC,2,wt,True,True,True,AX:AX_10_MIC,True,True,https://dx.doi.org/10.1128%2FmBio.01276-18
SRX3995806,2018-04-29 10:56:12,83332,Mycobacterium tuberculosis H37Rv,1773,1760,83332,Mycobacterium tuberculosis,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,AX_10_MIC,2,wt,True,True,True,AX:AX_10_MIC,True,True,https://dx.doi.org/10.1128%2FmBio.01276-18
SRX3995809,2018-04-29 10:56:12,83332,Mycobacterium tuberculosis H37Rv,1773,1760,83332,Mycobacterium tuberculosis,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,AX_30_MIC,2,wt,True,True,True,AX:AX_30_MIC,True,True,https://dx.doi.org/10.1128%2FmBio.01276-18
SRX3995808,2018-04-29 10:56:12,83332,Mycobacterium tuberculosis H37Rv,1773,1760,83332,Mycobacterium tuberculosis,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,AX_30_MIC,2,wt,True,True,True,AX:AX_30_MIC,True,True,https://dx.doi.org/10.1128%2FmBio.01276-18
SRX4619048,2019-10-31 17:06:04,83332,Mycobacterium tuberculosis H37Rv,1773,1760,83332,Mycobacterium tuberculosis,SINGLE,ILLUMINA,Illumina HiSeq 2500,...,QZ_10_MIC,2,wt,True,True,True,AX:QZ_10_MIC,True,True,https://dx.doi.org/10.1128%2FmBio.01276-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRX4498338,2018-09-25 17:56:05,1773,Mycobacterium tuberculosis,1773,1760,1773,Mycobacterium tuberculosis,SINGLE,ILLUMINA,NextSeq 500,...,1_days_176,3,0_days_control,True,True,True,vera:1_days_176,True,True,https://doi.org/10.1128/mbio.02627-19
SRX4498329,2018-09-25 17:56:05,1773,Mycobacterium tuberculosis,1773,1760,1773,Mycobacterium tuberculosis,SINGLE,ILLUMINA,NextSeq 500,...,1_days_control,4,0_days_control,True,True,True,vera:1_days_control,True,True,https://doi.org/10.1128/mbio.02627-19
SRX4498325,2018-09-25 17:56:05,1773,Mycobacterium tuberculosis,1773,1760,1773,Mycobacterium tuberculosis,SINGLE,ILLUMINA,NextSeq 500,...,1_days_control,4,0_days_control,True,True,True,vera:1_days_control,True,True,https://doi.org/10.1128/mbio.02627-19
SRX4498327,2018-09-25 17:56:05,1773,Mycobacterium tuberculosis,1773,1760,1773,Mycobacterium tuberculosis,SINGLE,ILLUMINA,NextSeq 500,...,1_days_control,4,0_days_control,True,True,True,vera:1_days_control,True,True,https://doi.org/10.1128/mbio.02627-19


In [20]:
# Nothing to change for the sample table -- the 0th column is the correct one to use.
# However, you can optionally suppress a warning about this from later on if you name the index:
ica_data.sample_table.index.name = 'sample'

# Also, there is an error in the existing 'Biological Replicates' column.
# This caused a "cannot set a row with mismatched columns" error in export (step 5).
# The easiest way to fix it is to have the code re-compute this column in step 4b.
# to make that happen, delete the existing one now.
#ica_data.sample_table = ica_data.sample_table.drop('Biological Replicates', axis = 1)

#### 4.b: Fill in the unknowns (Optional)

This step is already integrated in step 5 -- feel free to skip it. However, it might be a useful demonstration.

Even if you only have the minimum (M, A, X, and sample_table) data, calling the compatibility function with write=True will fill in enough blanks to generate the dataset. In this example, we need the code to fix the things that we ignored during step 4a.

In general, you may want to deepcopy the data before doing this. While the default columns shouldn't be an issue, the function requires that all iModulon names be numbers, which might not be desired for you. Under the hood in step 5, we automatically deep copy your data before changing anything. If you want access to the default values and columns, however, you may be interested in doing this without the deepcopy.

In [21]:
from copy import deepcopy

new_data = deepcopy(ica_data)

imodulondb_compatibility(new_data, inplace = True)

iModulon Table is missing a Regulator column
Note that Regulator is displayed in the dataset page so it can have spelled out 'and' and 'or' operators.If you have a TF column, a Regulator column will begenerated for you (but not vice versa).

Gene Table is missing the optional length column



Let's call it again just to see what the output is now that the new elements are written:

In [22]:
imodulondb_compatibility(new_data)


Gene Table is missing the optional length column



A very minimal output is good!

## 5: Export

You are now ready to export! This process may take some time (up to around 1 hr for PRECISE2, but significantly shorter depending on the number of genes your organism has).

The export function needs a path to export to. This should be path to where you cloned the iModulonDB repository.

The function will first deepcopy your files and then call the compatibility function from above with write = True. Then it will generate all files that don't require iteration, then all iModulon files, then all gene files.

As it goes, it will print anything you may want to be aware of. This is where issues with the TRN may become apparent.

In [23]:
imodulondb_export(ica_data, '../iModulonDB/')

iModulon Table is missing a Regulator column
Note that Regulator is displayed in the dataset page so it can have spelled out 'and' and 'or' operators.If you have a TF column, a Regulator column will begenerated for you (but not vice versa).

Gene Table is missing the optional length column

Writing main site files...
Two progress bars will appear below. The second will take significantly longer than the first.


TF not in TRN: lsyG
To remedy this, add rows to the TRN for each gene associated with this TF
TF has no associated expression profile: lsyG
If lsyG is not a gene, this behavior is expected.
If it is a gene, use consistent naming between the TRN and gene_table.



## 6: Update TRN, other gene info if desired

In the output above, you will notice snippets such as this one:

> TF has no associated expression profile: cecR

>If cecR is not a gene, this behavior is expected.

>If it is a gene, use consistent naming between the TRN and gene_table.

If desired, you could look up each of these to find alternative names that may be used, then rename either the TRN (all instances of the name in the regulator column) or the gene_table (the name column in the row with the appropriate locus) so that they all match. This will not need to be done for every case; for example, 'ile-tRNA' is a leucine-bound tRNA molecule, which does not have an associated gene.

After doing this, you would need to rerun the export function (step 5).

In [24]:
# according to EcoCyc, the b-number for cecR is b0796 and its alternative name is 'ybiH'

#ica_data.gene_table.loc['b0796']

In [25]:
# To remedy this matchup, I will replace 'ybiH' with 'cecR' in the gene_table.

#ica_data.gene_table.loc['b0796', 'gene_name'] = 'cecR'

# repeat this process for all unmatched genes.

# re-running step 5 would then correct this error.

## 7: Paste HTML

This step only needs to occur once! Time to add the link to the splash page.

1. Navigate to your iModulonDB/organisms/M_favoritespeciesus/modulome folder.
2. Open the file 'html_for_splash.html' in a text editor
3. Copy the entire contents to your clipboard
4. Navigate back to the main repository folder
5. Open 'index.html' in a text editor
6. Scroll down to the comment near line 250 about "NEW DATASETS"
7. Paste in the appropriate location. You may want to adjust the indentations.
8. Save the file
9. Enjoy your new website!

Traveling to localhost:8000 in your browser should show you the site with a shiny new button for your dataset. Click it to see your new iModulons.

## Moving Forward

Any time you make a change that you want reflected on your site, simply repeat step 5! If changes don't immediately appear on the browser, try a force refresh (Ctrl+Shift+R in Google Chrome).

## Don't forget to save your file so that step 5 is all that is required in the future!

In [26]:
save_to_json(ica_data, 'mtb_data.json')